In [28]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

Detection

In [29]:
sg_tsv = './features_detection/training/test.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})

In [30]:
sg_tsv = './features_detection/training/test2.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [31]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
#full_detection

In [32]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:03, 301.15it/s]


Segmentation

In [33]:
sg_tsv = './features_test1/training/test.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})

In [34]:
sg_tsv = './features_test1/training/test.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [35]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...,"[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...,"[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...,"[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...,"[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...,"[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...,"[{'class': 'Tennis racket', 'conf': 0.73602569..."
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...,"[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...,"[{'class': 'Skirt', 'conf': 0.9404543042182922..."
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...,"[{'class': 'Ring', 'conf': 0.5740575194358826,..."


In [36]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:01, 695.96it/s]


Union

In [37]:
full_segmentation["union"] = full_segmentation["label"] + full_detection["label"]
full_segmentation 

,id,features,label,union
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...,"[{'class': 'Earring', 'conf': 0.97952914237976...","[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...,"[{'class': 'Dog', 'conf': 0.9183911085128784, ...","[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...,"[{'class': 'Belt', 'conf': 0.6726886630058289,...","[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...,"[{'class': 'Goggles', 'conf': 0.94563788175582...","[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...,"[{'class': 'Jean', 'conf': 0.7828388214111328,...","[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...,"[{'class': 'Tennis racket', 'conf': 0.73602569...","[{'class': 'Tennis racket', 'conf': 0.73602569..."
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...,"[{'class': 'Hat', 'conf': 0.7257902026176453, ...","[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...,"[{'class': 'Skirt', 'conf': 0.9404543042182922...","[{'class': 'Skirt', 'conf': 0.9404543042182922..."
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...,"[{'class': 'Ring', 'conf': 0.5740575194358826,...","[{'class': 'Ring', 'conf': 0.5740575194358826,..."


In [38]:
for i in tqdm(range(len(full_segmentation))):
    if(len(full_segmentation["label"].iloc[i]) + len(full_detection["label"].iloc[i]) != len(full_segmentation["union"].iloc[i])):
        print("!!!!!!!!!!!!!!!")

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 20831.12it/s]


In [39]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [40]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [41]:
full_segmentation['feature_union'] = full_segmentation.progress_apply(generate_features,axis=1)
full_segmentation['feature_union'] = full_segmentation['feature_union'].progress_apply(json.dumps)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 229.81it/s]


In [42]:
full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2542.17it/s]


In [43]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_full/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'test.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'test.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [44]:
sg_tsv = './training_union_full/test.hw.tsv'

df_train_hw = pd.read_csv(sg_tsv,sep='\t',header = None,converters={1:json.loads})#converters={1:ast.literal_eval})
df_train_hw = df_train_hw.rename(columns={0: "id", 1: "dim"})

df_train_hw

,id,dim
0,1007129816,"[{'height': 461, 'width': 500}]"
1,1009434119,"[{'height': 333, 'width': 500}]"
2,101362133,"[{'height': 500, 'width': 347}]"
3,102617084,"[{'height': 375, 'width': 500}]"
4,10287332,"[{'height': 375, 'width': 500}]"
...,...,...
995,900144365,"[{'height': 385, 'width': 500}]"
996,94024624,"[{'height': 332, 'width': 500}]"
997,95758790,"[{'height': 333, 'width': 500}]"
998,97233789,"[{'height': 333, 'width': 500}]"


In [45]:
tsv_writer(df_train_hw.merge(full_segmentation, on=["id"])[["id","feature_union"]].values.tolist(), FEATURE_FILE)

In [46]:
tsv_writer(df_train_hw.merge(full_segmentation, on=["id"])[["id","label_union"]].values.tolist(), LABEL_FILE)

In [ ]:
# test

In [52]:
sg_tsv = './training_union_full/test.feature.tsv'
df_features = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features = df_features.rename(columns={0: "id", 1: "feature"})
df_features

,id,feature
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...
...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...


In [53]:
sg_tsv = './training_union_full/test.label.tsv'
df_labels = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels = df_labels.rename(columns={0: "id", 1: "label"})
df_labels

,id,label
0,1007129816,"[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,"[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,"[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,"[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,"[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...
995,900144365,"[{'class': 'Tennis racket', 'conf': 0.73602569..."
996,94024624,"[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,"[{'class': 'Skirt', 'conf': 0.9404543042182922..."
998,97233789,"[{'class': 'Ring', 'conf': 0.5740575194358826,..."


In [54]:
for elem in tqdm(df_features["feature"].values): # check
    if len(np.frombuffer(base64.b64decode(elem["features"]), np.float32)) / elem["num_boxes"] != 2054:
        print("noooooooooooooooo!!!!!!!")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 326.47it/s]


In [55]:
#df_labels.merge(df_features, on=["id"])["feature"].iloc[0]
for elem, label in tqdm(zip(df_labels.merge(df_features, on=["id"])["feature"].values, df_labels.merge(df_features, on=["id"])["label"].values)): # check
    if elem["num_boxes"] != len(label):
        print("noooooooooooooooo!!!!!!!")#num_boxes

1000it [00:00, ?it/s]


In [56]:
df_labels.merge(df_features, on=["id"])["feature"].iloc[0]["num_boxes"] == len(df_labels.merge(df_features, on=["id"])["label"].iloc[0])

True